# Interpolate elevation data from the spline fit

In [17]:
from shapely.ops import nearest_points, LineString, Point
from pathlib import Path
import geopandas as gpd
import rasterio

import matplotlib.pyplot as plt
import numpy as np
import json

import src.elevation_tools as elevation_tools

import sys
sys.path.insert(0,str(Path.cwd().parent))
import file_structure_setup
config = file_structure_setup.filepaths()

In [2]:
#for storing the interpolated points with sampled elevation data
import pickle
with (config['network_fp']/'spline_fit_elevation.pkl').open('rb') as fh:
    interpolated_points_dict = pickle.load(fh)

In [4]:
raw_links = gpd.read_file(config['osmdwnld_fp'] / f"osm_{config['geofabrik_year']}.gpkg",layer="raw")
#set the osmid as the index
raw_links.set_index('osmid',inplace=True)
#raw_links = raw_links[['oneway','geometry']]

In [5]:
links = gpd.read_file(config['network_fp']/'networks.gpkg',layer='osm_links')

In [14]:
dem_urls = elevation_tools.get_dem_urls(links.to_crs('epsg:4326'))
dem_crs = rasterio.open(dem_urls[0]).crs
links.to_crs(dem_crs,inplace=True)
raw_links.to_crs(dem_crs,inplace=True)

Example

In [ ]:
# #select a link and try it
# linkid = 637636161
# link = links[links['osmid']==linkid].iloc[[0],:]

# #get osm line
# line = raw_links.loc[linkid,'geometry']#interpolated_points_dict[linkid]['geometry']
# line = np.array(line.coords)

# #get geo of start and end
# #or just use the included line to reduce memory?
# pointA = nodes[nodes['osm_N']==link['osm_A'].item()]
# pointB = nodes[nodes['osm_N']==link['osm_B'].item()]
# print(line)

In [ ]:
# # Define the coordinates of two additional points
# point1 = (pointA.geometry.item().x,pointA.geometry.item().y)
# point2 = (pointB.geometry.item().x,pointB.geometry.item().y)
# print(point1,point2)

In [ ]:
# # Plot the GeoDataFrame and the additional points
# fig, ax = plt.subplots()

# # plot the full link
# ax.plot(line[:,0],line[:,1], color='gray', label='full osm')

# link.plot(ax=ax, color='blue', label='osm segemnt')
# ax.plot(point1[0], point1[1], marker='o', color='red', markersize=10, label='Point 1')
# ax.plot(point2[0], point2[1], marker='o', color='green', markersize=10, label='Point 2')

# # Add labels to the additional points
# #ax.text(point1[0], point1[1], 'Point 1', fontsize=12, ha='right')
# #ax.text(point2[0], point2[1], 'Point 2', fontsize=12, ha='right')

# # Add legend and labels
# ax.legend()
# ax.set_xlabel('Longitude')
# ax.set_ylabel('Latitude')
# ax.set_title('GeoDataFrame with Additional Points')

# # Manually set limits to create a square aspect ratio
# min_x, max_x = ax.get_xlim()
# min_y, max_y = ax.get_ylim()
# width = max(max_x - min_x, max_y - min_y)
# center_x = (min_x + max_x) / 2
# center_y = (min_y + max_y) / 2
# ax.set_xlim(center_x - width / 2, center_x + width / 2)
# ax.set_ylim(center_y - width / 2, center_y + width / 2)

# plt.show()


In [ ]:
# point1_geo = Point(point1)
# point2_geo = Point(point2)
# line_geo = LineString(line)

Find the distance of the shapepoint on each line

In [ ]:
# from shapely import line_locate_point, equals_exact

# point1_dist = line_locate_point(LineString(line),Point(point1))
# point2_dist = line_locate_point(LineString(line),Point(point2))

# #scenario 1: last point intersects with early point on a line (line loops into itself)
# #so trim off the points before point 1
# if point1_dist >= point2_dist:
#     for first_i, point in enumerate(line):
#         if equals_exact(Point(point),Point(point1),tolerance=1):
#             break
#     new_line = line[first_i+1:]
#     point2_dist = line_locate_point(LineString(new_line),Point(point1))

# #scenario 2: first point intersect with last point on a line
# #so trim off the point at the end of the line
# if point1_dist >= point2_dist:
#     new_line = line[0:-1]
#     point1_dist = line_locate_point(LineString(new_line),Point(point1))
#     point2_dist = line_locate_point(LineString(line),Point(point1))
    
# if point1_dist >= point2_dist:
#     print('error')
# else:
#     print(np.round(point1_dist),np.round(point2_dist))

# Interpolate distance on line
For each link, we need the distance along the line of the start and end point to properly input into the fitted spline

For polygons where a way intersects with itself, we may need some more advanced logic utilizing the node sequence. 

If we have the node sequence, start with the first node given, and see if the second node can be found after that. If it can't then we know we need to reverse the direction, but can we think of an example where 

In [18]:
from shapely import line_locate_point, equals_exact
from shapely.ops import polygonize, unary_union
from itertools import product

interpolated_distance_dict = {}

for idx, row in links.iterrows():
    
    #network link attributes
    osmid = row['osmid']
    a = row['osm_A']
    b = row['osm_B']
    network_line = np.array(row['geometry'].coords)
    point1 = network_line[0]
    point2 = network_line[-1]

    #get raw osm attributes
    line = np.array(raw_links.loc[osmid,'geometry'].coords)
    points = [Point(x) for x in line]
    points = gpd.GeoSeries(points)
    distances = points.distance(points.shift(1)).cumsum().tolist()
    distances[0] = 0
    node_list = json.loads(raw_links.loc[osmid,'all_tags'])['@way_nodes']
    a_idx = node_list.index(a)
    b_idx = node_list.index(b)

    #polygon check
    poly_check = len(list(polygonize(unary_union(LineString(line))))) #> 0
    poly_check_network = len(list(polygonize(unary_union(LineString(network_line))))) #> 0

    method = 'none'
    reverse_geometry = False
    
    # scenario 1: simple line segment no loops
    if (poly_check == 0) & (poly_check_network == 0) & (a != b):
        method = 'simple'
        # step 1: check if network line is in the same direction as loop
        if (a_idx > b_idx):    
            reverse_geometry = True
            a = row['osm_B']
            b = row['osm_A']
            a_idx = node_list.index(a)
            b_idx = node_list.index(b)

        # step 2: get distance of start and end point along the line
        #point1_dist = line_locate_point(LineString(line),Point(point1))
        #point2_dist = line_locate_point(LineString(line),Point(point2))
        network_dist = distances[a_idx:b_idx+1]
        point1_dist = network_dist[0]
        point2_dist = network_dist[-1]

        # step 3: check to see if distance matches the network geometry
        difference = point2_dist - point1_dist
        distance_check = np.abs(difference - LineString(network_line).length) < 0.02
        if distance_check == False:
            print('error')

        segment = [point1_dist,point2_dist]
    
    # scenario 2: network segment is not a loop but the line is
    elif (poly_check > 0) & (poly_check_network == 0) & (a != b):
        method = 'simple loop'

        a_s = [idx for idx, val in enumerate(node_list) if val == a]
        b_s = [idx for idx, val in enumerate(node_list) if val == b]

        #get all possible combinations of A/B
        combinations = list(product(a_s,b_s))
        for a_idx, b_idx in combinations:
            # step 1: see if network line direction matches line
            if (a_idx > b_idx):
                reverse_geometry = True
                # a = row['osm_B']
                # b = row['osm_A']
                # a_idx = node_list.index(a)
                # b_idx = node_list.index(b)
                a_idx, b_idx = b_idx, a_idx

            # step 2: get distance of start and end point along the line
            # start = node_list.index(a)
            # end = node_list[start+1:].index(b) + start + 1
            network_dist = distances[a_idx:b_idx+1]
            point1_dist = network_dist[0]
            point2_dist = network_dist[-1]

            # step 3: check to see if distance matches the network geometry
            difference = point2_dist - point1_dist
            distance_check = np.abs(difference - LineString(network_line).length) < 0.02
            
            segment = [point1_dist,point2_dist]

            if distance_check == True:
                break

        # step 4: if it doesn't then it is likely that the network segment crosses the loop
        # we'll need multiple segments in that case
        if distance_check == False:
            method = 'complex loop'

            #reset the reversed geometry bit
            reverse_geometry = False
            a = row['osm_A']
            b = row['osm_B']
            a_idx = node_list.index(a)
            b_idx = node_list.index(b)

            # if the network line crosses the loop then b_idx will be larger
            if a_idx < b_idx:
                reverse_geometry = True
                a = row['osm_B']
                b = row['osm_A']
                a_idx = node_list.index(a)
                b_idx = node_list.index(b)

            # step 5: get the two segments
            first_segment = [0,distances[b_idx]]
            second_segment = [distances[a_idx],distances[-1]]

            # step 6: check distance
            difference = second_segment[-1] - second_segment[0] + first_segment[1]
            distance_check = np.abs(difference - LineString(network_line).length) < 0.02

            # if distance_check == False:
            #     print('error')
            segment = [first_segment,second_segment]

    else:
        method = 'none'
        segment = 999
        distance_check = False
    
    #finaly assemble the dict
    interpolated_distance_dict[idx] = {
    'method': method,
    'segment': segment,
    'reverse_geometry': reverse_geometry,
    'distance_check': distance_check,
    'poly_check': poly_check
    }

error
error


In [19]:
import pandas as pd
df = pd.DataFrame.from_dict(interpolated_distance_dict,orient='index')
interpolated = pd.merge(links,df,left_index=True,right_index=True)

#drop loops
interpolated = interpolated[interpolated['method']!='none']

In [20]:
interpolated

,osm_A,osm_B,osm_linkid,link_type,osmid,geometry,method,segment,reverse_geometry,distance_check,poly_check
0,436633256,67075136,1125565548,restricted_access_road,37392645,"LINESTRING (462168.790 3555339.363, 462786.676...",simple,"[0, 3985.052055066325]",False,True,0
1,67075036,68560350,1125565549,restricted_access_road,9199408,"LINESTRING (465533.732 3553235.043, 465191.859...",simple,"[0, 3987.3050020401697]",False,True,0
2,68527352,68527407,1125565550,road,9196359,"LINESTRING (464762.654 3554690.446, 464775.451...",simple,"[0, 858.561165737886]",False,True,0
3,68530452,68527407,1125565551,road,9196794,"LINESTRING (465698.840 3554048.992, 465684.399...",simple,"[528.4724599190748, 1031.8289339128569]",False,True,0
4,68527407,68527414,1125565552,road,9196359,"LINESTRING (465559.288 3554511.142, 465583.341...",simple,"[858.561165737886, 1867.6742661193555]",False,True,0
...,...,...,...,...,...,...,...,...,...,...,...
129622,10277959062,10277959063,1125695170,parking_and_driveways,1123883742,"LINESTRING (489518.021 3548376.460, 489584.089...",simple,"[0, 129.18201097358696]",False,True,0
129623,10277959064,10277959065,1125695171,parking_and_driveways,1123883743,"LINESTRING (489534.631 3548366.644, 489600.983...",simple,"[0, 129.33635019608826]",False,True,0
129624,10277959066,10277959067,1125695172,parking_and_driveways,1123883744,"LINESTRING (489556.241 3548353.864, 489622.790...",simple,"[0, 129.54266507004127]",False,True,0
129625,10277959068,10277959069,1125695173,parking_and_driveways,1123883745,"LINESTRING (489573.436 3548343.704, 489639.222...",simple,"[0, 129.67761703112774]",False,True,0


In [ ]:
#todo create a checkpoint here

In [ ]:
#export['segment'] = export['segment'].astype(str)
#export['length_m'] = export.length
#export.to_file(Path.home()/'Downloads/scratch.gpkg',layer='all')

# Spline fit
We need elevation data between the two points to interpolate an elevation profile for the smaller links

In [21]:
from scipy.interpolate import splrep, splev, BSpline
import src.elevation_tools as elevation_tools

interpolate_dist_m = 1
new_elevations_dict = {}
spline_or_nah = []

for idx, row in interpolated.iterrows():
    #get ids
    osm_linkid = row['osm_linkid']
    osmid = row['osmid']

    #get osm elevation
    item = interpolated_points_dict[osmid]

    #retrieve the fitted spline if it exists
    spline = item.get('spline',0)
    
    #get segments
    segment = row['segment']
    
    #if no spline, do linear interpolatation
    if spline == 0:
        new_xs = item['distances']
        new_elevations = item['elevations']
        spline_or_nah.append(idx)

    elif isinstance(segment[0],list):
        #get x sequence
        new_xs_segment1 = np.arange(int(segment[0][0]),int(segment[0][1])+interpolate_dist_m,interpolate_dist_m)
        new_xs_segment2 = np.arange(int(segment[1][0]),int(segment[1][1])+interpolate_dist_m,interpolate_dist_m)

        #get new elevations
        new_elevations_segment1 = splev(new_xs_segment1,spline)
        new_elevations_segment2 = splev(new_xs_segment2,spline)

        #combine
        new_xs = np.arange(0,len(new_xs_segment1)+len(new_xs_segment2))
        new_elevations = np.hstack([new_elevations_segment2,new_elevations_segment1])

    else:
        #get new elevation values
        new_xs = np.arange(int(segment[0]),int(segment[1])+interpolate_dist_m,interpolate_dist_m)
        new_elevations = splev(new_xs, spline)
        #recalculate elevations stats
        #TODO if wanting to seperate grades by section use the elevation_stats function
    
    new_elevations_dict[idx] = elevation_tools.simple_elevation_stats(new_xs, new_elevations)


/Users/tannerpassmore/Documents/GitHub/BikewaySimDev/network/src/elevation_tools.py:298: RuntimeWarning: invalid value encountered in scalar divide
  ascent_grade = np.round(ascent / total_distance * 100,2)
/Users/tannerpassmore/Documents/GitHub/BikewaySimDev/network/src/elevation_tools.py:299: RuntimeWarning: invalid value encountered in scalar divide
  descent_grade = np.round(descent / total_distance * 100,2)


In [22]:
interpolated.loc[spline_or_nah,'spline_fit'] = False

In [23]:
df = pd.DataFrame.from_dict(new_elevations_dict,orient='index')
elevations_added = pd.merge(interpolated,df,left_index=True,right_index=True)

In [24]:
elevations_added['segment'] = elevations_added['segment'].astype(str)
elevations_added.to_file(config['network_fp']/'elevation.gpkg')

In [25]:
elevations_added.head()

,osm_A,osm_B,osm_linkid,link_type,osmid,geometry,method,segment,reverse_geometry,distance_check,poly_check,spline_fit,ascent_m,descent_m,ascent_grade_%,descent_grade_%
0,436633256,67075136,1125565548,restricted_access_road,37392645,"LINESTRING (462168.790 3555339.363, 462786.676...",simple,"[0, 3985.052055066325]",False,True,0,NaN,0.2,-2.2,0.01,-0.05
1,67075036,68560350,1125565549,restricted_access_road,9199408,"LINESTRING (465533.732 3553235.043, 465191.859...",simple,"[0, 3987.3050020401697]",False,True,0,NaN,1.9,-0.0,0.05,-0.00
2,68527352,68527407,1125565550,road,9196359,"LINESTRING (464762.654 3554690.446, 464775.451...",simple,"[0, 858.561165737886]",False,True,0,NaN,1.9,-3.7,0.22,-0.43
3,68530452,68527407,1125565551,road,9196794,"LINESTRING (465698.840 3554048.992, 465684.399...",simple,"[528.4724599190748, 1031.8289339128569]",False,True,0,NaN,1.0,-1.5,0.10,-0.14
4,68527407,68527414,1125565552,road,9196359,"LINESTRING (465559.288 3554511.142, 465583.341...",simple,"[858.561165737886, 1867.6742661193555]",False,True,0,NaN,3.6,-4.3,0.19,-0.23


In [28]:
elevations_added[['ascent_grade_%','descent_grade_%']].abs().max()

ascent_grade_%     94.57
descent_grade_%    47.90
dtype: float64

In [32]:
(elevations_added[['ascent_grade_%','descent_grade_%']] > 5).sum()

ascent_grade_%     1536
descent_grade_%       0
dtype: int64

In [35]:
elevations_added[(elevations_added[['ascent_grade_%','descent_grade_%']] > 5).any(axis=1)]

,osm_A,osm_B,osm_linkid,link_type,osmid,geometry,method,segment,reverse_geometry,distance_check,poly_check,spline_fit,ascent_m,descent_m,ascent_grade_%,descent_grade_%
42,66519886,3475433945,1125565590,road,340328545,"LINESTRING (463675.885 3549316.364, 463643.981...",simple,"[0, 100.89200288396933]",False,True,0,NaN,5.5,-0.2,5.50,-0.21
62,67123582,67123575,1125565610,road,340328551,"LINESTRING (464189.274 3549456.832, 464138.718...",simple,"[0, 112.41023442089417]",False,True,0,NaN,5.6,-5.3,5.03,-4.73
372,7601217911,10259710937,1125565920,parking_and_driveways,1121724870,"LINESTRING (469250.697 3549962.218, 469240.909...",simple,"[0, 354.7336053452333]",False,True,0,NaN,18.5,-1.3,5.24,-0.37
381,9300366649,9300366650,1125565929,service,1008101635,"LINESTRING (468810.919 3549971.588, 468756.044...",simple,"[0, 56.87483598046642]",False,True,0,NaN,4.5,0.0,8.03,0.00
494,68527474,68527477,1125566042,no_access_or_private,9196363,"LINESTRING (469023.194 3556209.620, 469003.661...",simple,"[0, 37.02626984351419]",False,True,0,NaN,2.0,-0.3,5.44,-0.91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128861,8868813016,8868836217,1125694409,service,958515246,"LINESTRING (492165.261 3538037.005, 492167.760...",simple,"[0, 17.240916592489775]",False,True,0,False,0.6,0.0,6.00,0.00
128862,8868801236,8868801230,1125694410,service,958511484,"LINESTRING (492134.724 3538046.416, 492138.190...",simple,"[0, 10.841261541405682]",False,True,0,False,0.6,0.0,6.00,0.00
129123,67075334,67075338,1125694671,restricted_access_road,37393174,"LINESTRING (489908.268 3548225.605, 489834.179...",simple,"[0, 91.76176362840339]",False,True,0,NaN,6.7,-8.0,7.36,-8.77
129589,10277959046,10277959052,1125695137,service,1123883737,"LINESTRING (489660.899 3548334.115, 489627.251...",simple,"[0, 38.160431993170434]",False,True,0,NaN,4.2,-0.9,11.06,-2.26


In [29]:
elevations_added[['ascent_grade_%','descent_grade_%']].abs().describe()

,ascent_grade_%,descent_grade_%
count,129622.000000,129622.000000
mean,0.613465,0.314435
std,1.379218,0.985363
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.040000,0.010000
75%,0.600000,0.220000
max,94.570000,47.900000


In [ ]:
# from scipy.interpolate import splrep, splev, BSpline

# # case 1: only one segment to deal with
# osm_linkid = 1125605250
# osmid = 751119047

# row = export[export['osm_linkid']==osm_linkid].squeeze()

# #get osm elevation
# item = interpolated_points_dict[osmid]

# #retrieve the fitted spline
# spline = item['spline']

# #get segments
# segment = row['segment']

# #get x sequence
# new_xs_segment1 = np.arange(int(segment[0][0]),int(segment[0][1])+interpolate_dist_m,interpolate_dist_m)
# new_xs_segment2 = np.arange(int(segment[1][0]),int(segment[1][1])+interpolate_dist_m,interpolate_dist_m)

# #get new elevations
# new_elevations_segment1 = splev(new_xs_segment1,spline)
# new_elevations_segment2 = splev(new_xs_segment2,spline)

# #combine
# new_xs = np.arange(0,len(new_xs_segment1)+len(new_xs_segment2))
# new_elevations = np.hstack([new_elevations_segment2,new_elevations_segment1])

# fig, ax = plt.subplots()
# ax.plot(item['distances'],item['elevations'],'-')
# ax.plot(new_xs_segment1,new_elevations_segment1,'-.')
# ax.plot(new_xs_segment2,new_elevations_segment2,'-.')

In [ ]:
# # use fitted spline to from dict to create new values

# item = interpolated_points_dict[osmid]
# spline = interpolated_points_dict[osmid]['spline']

# interpolate_dist_m = 1
# new_xs = np.arange(int(point1_dist),int(point2_dist)+interpolate_dist_m,interpolate_dist_m)
# new_elevations = splev(new_xs, spline)

#new_grades = pd.Series(new_elevations).diff() #/ interpolate_dist_m * 100

In [ ]:
# from importlib import reload
# import src.elevation_tools as elevation_tools
# reload(elevation_tools)
# elevation_tools.simple_elevation_stats(new_xs, new_elevations)
# fig, ax = plt.subplots()
# ax.plot(item['distances'],item['elevations'],'-')
# ax.plot(new_xs,new_elevations,'-.')

In [ ]:
# linkid
# item = interpolated_points_dict[linkid]
# import numpy as np
# from scipy.interpolate import splrep, splev, BSpline

# spline = interpolated_points_dict[linkid]['spline']

# new_xs = np.arange(int(point1_dist),int(point2_dist)+10,10)
# new_ys = splev(new_xs, spline)
# new_ys
# fig, ax = plt.subplots()
# ax.plot(item['distances'],item['elevations'],'-')
# ax.plot(new_xs,new_ys,'-.')

Add elevation data to links (deprecated)

In [ ]:
# grade_cats = {}

# for linkid, item in tqdm(interpolated_points_dict.items()):     

#     outputs = elevation_tools.elevation_stats(item['distances'],item['smoothed'],grade_threshold)

#     df = pd.DataFrame({
#         'distance_deltas':outputs['distance_deltas'],
#         'segment_grades':outputs['segment_grades']
#         })
#     #create bins (broach 2012 ones)
#     #'(-inf,-6]','(-6,-4]','(-4,-2]','(-2,0]',
#     #-np.inf,-6,-4,-2,
#     bins = [-1,2,4,6,10,15,np.inf]
#     names = ['(0,2]','(2,4]','(4,6]','(6,10]','(10,15]','(15,inf]']
#     df['grade_category'] = pd.cut(df['segment_grades'].abs(), bins, labels = names)

#     #determine if up or down (doesn't matter if flat)
#     df.loc[df['segment_grades'] >= 0,'ascent_or_descent'] = 'ascent'
#     df.loc[df['segment_grades'] < 0,'ascent_or_descent'] = 'descent'

#     # how many meters at each grade category?
#     test = df.groupby(['grade_category','ascent_or_descent'],observed=False)['distance_deltas'].sum().round(0)
#     test = pd.DataFrame(test).transpose()
#     test.index = [linkid]
#     grade_cats[linkid] = {
#         'ascent_m': outputs['ascent'],
#         'descent_m': outputs['descent'],
#         'ascent_grade': outputs['ascent_grade'],
#         'descent_grade': outputs['descent_grade']
#     }
#     grade_cats[linkid].update(
#         test.to_dict(orient='records')[0]
#     )
    
# #.items())#.reseorient='t_index()#.pivot(
#     # columns=['grade_category','up'],
#     # values='distance_deltas'
#     # )
# #grade_cats